In [1]:
using ElectricGrid
using PlotlyJS

WebIO._IJuliaInit()

## The PV-Array

This notebook will explain the PV arrays structurally and how they were implemented.

As a basis serves the [paper](https://doi.org/10.1016/j.nrjag.2014.04.001) which explains the physical characteristics of a PV module in more detail. In order to avoid further calculations at runtime, a simpler approximation is used, the ideal single diode model. This neglects the parallel and series resistance of the PV module.

In the following we define a mutable struct with the most important data of a PV module, most of the data are given by the data sheets.

In [2]:
ModulePV = SolarModule(); # For this example we use the default values

As a rule, many of the modules are initially connected in series, especially in large systems, in order to increase the voltage. Further modules can then be added in parallel to further increase the current.

We therefore define another mutable struc, which inherits the properties of the PV module and specifies how many modules are to be created in series and then again in parallel to it.

For real applications, this often leads to problems, since shadowing of individual cells (!!!) can result in a large drop in power. Therefore, in practice, countermeasures are often taken, such as the integration of bypass diodes.

In [19]:
ArrayPV = SolarArray(;ModuleParameters=ModulePV, serial=1, parallel=1)

SolarArray(SolarModule(2.0381e-10, 1.2, 1.3806e-23, 1.6022e-19, 273, 36, 1000, 0.0013, 3.11), 1, 1)

Now let us take a look  characteristics for different irradiations. Therefore, we define a function that gives us the current as a function of voltage, irradiation and temperature. In addition, a function should also output the voltage as a function of current, irradiation and temperature, since this will later be interesting for our application in ElectricGrid.jl.

### Funktion for module

For the purpose of this notebook, the functions here are defined vectorially so that they can be evaluated directly for multiple irradiances and voltages.

The functions that are later used in the ElectricGrid.jl environment return only a scalar value.

In [20]:
function GetI_vec(SolarArr::SolarArray, V, G, T)
    self = SolarArr.ModuleParameters
    
    function I_photo(self::SolarModule, G, T)
        dT = self.T_0 + T
        I_ph = G./self.G_ref*(self.I_ph_ref + self.mu_sc * dT).* ones(length(V))'
        return I_ph
    end;

    function I_diode(self::SolarModule, V, G, T)
        dT = self.T_0 + T
        V_T = self.k*dT/self.q
        I_d = self.I_0 * (exp.(V  / (self.ni * self.N_cell * SolarArr.serial * V_T)).-1).*ones(5)'
        return I_d
    end;
    
    I = (I_photo(self, G, T)' - I_diode(self, V, G, T)) * SolarArr.parallel
    return I
end


GetI_vec (generic function with 1 method)

In [21]:
# Not working!

function GetV_vec(self::SolarModule, I, G, T)
    function I_photo(self::SolarModule, G, T)
        dT = self.T_0 + T
        I_ph = G/self.G_ref*(self.I_ph_ref + self.mu_sc * dT)* ones(length(I))';
        return I_ph
    end;
    
#     I = maximum.([0,I])
    
    res = (I_photo(self, G, T)-I)
    
    dT = self.T_0 + T
    V_T = self.k*dT/self.q
    
    if res <= 0
        V=0
    else
        V = self.ni*self.N_cell*V_T*(log((res)/self.I_0)+1)
    end
    
    return V
end

GetV_vec (generic function with 1 method)

## Characteristics for PV modules

Here the values are defined for which the modules are to be evaluated.

In [22]:
T = 25
G = collect(200:200:1000)
V = collect(0:0.1:30);

In [23]:
i = GetI_vec(ArrayPV, V, G, T);

In [24]:
labels = ["200 W/m^2" "400 W/m^2" "600 W/m^2" "800 W/m^2" "1000 W/m^2"];
plot([scatter(x=V, y=i[:,1], mode="lines", name=labels[1]),
scatter(x=V, y=i[:,2], mode="lines", name=labels[2]),
scatter(x=V, y=i[:,3], mode="lines", name=labels[3]),
scatter(x=V, y=i[:,4], mode="lines", name=labels[4]),
scatter(x=V, y=i[:,5], mode="lines", name=labels[5])],
Layout(xaxis_range=[0,30], yaxis_range=[0,5], title="PV module - I(V)", xaxis_title="V", yaxis_title="I"))

data: [
  "scatter with fields mode, name, type, x, and y",
  "scatter with fields mode, name, type, x, and y",
  "scatter with fields mode, name, type, x, and y",
  "scatter with fields mode, name, type, x, and y",
  "scatter with fields mode, name, type, x, and y"
]

layout: "layout with fields margin, template, title, xaxis, and yaxis"

In [25]:
V_ = V.*ones(5)';
P = i .* V_;

In [26]:
plot([
scatter(x=V, y=P[:,1], mode="lines", name=labels[1]),
scatter(x=V, y=P[:,2], mode="lines", name=labels[2]),
scatter(x=V, y=P[:,3], mode="lines", name=labels[3]),
scatter(x=V, y=P[:,4], mode="lines", name=labels[4]),
scatter(x=V, y=P[:,5], mode="lines", name=labels[5])],
Layout(xaxis_range=[0,30], yaxis_range=[0,100], title="PV module - P(V)", xaxis_title="V", yaxis_title="P"))

data: [
  "scatter with fields mode, name, type, x, and y",
  "scatter with fields mode, name, type, x, and y",
  "scatter with fields mode, name, type, x, and y",
  "scatter with fields mode, name, type, x, and y",
  "scatter with fields mode, name, type, x, and y"
]

layout: "layout with fields margin, template, title, xaxis, and yaxis"

In [27]:
plot([
scatter(y=V, x=i[:,1], mode="lines", name=labels[1]),
scatter(y=V, x=i[:,2], mode="lines", name=labels[2]),
scatter(y=V, x=i[:,3], mode="lines", name=labels[3]),
scatter(y=V, x=i[:,4], mode="lines", name=labels[4]),
scatter(y=V, x=i[:,5], mode="lines", name=labels[5])],
Layout(xaxis_range=[0,5], yaxis_range=[0,30], title="PV module - V(I)", xaxis_title="I", yaxis_title="V"))

data: [
  "scatter with fields mode, name, type, x, and y",
  "scatter with fields mode, name, type, x, and y",
  "scatter with fields mode, name, type, x, and y",
  "scatter with fields mode, name, type, x, and y",
  "scatter with fields mode, name, type, x, and y"
]

layout: "layout with fields margin, template, title, xaxis, and yaxis"

In [28]:
plot([
scatter(y=P[:,1], x=i[:,1], mode="lines", name=labels[1]),
scatter(y=P[:,2], x=i[:,2], mode="lines", name=labels[2]),
scatter(y=P[:,3], x=i[:,3], mode="lines", name=labels[3]),
scatter(y=P[:,4], x=i[:,4], mode="lines", name=labels[4]),
scatter(y=P[:,5], x=i[:,5], mode="lines", name=labels[5])],
Layout(xaxis_range=[0,5], yaxis_range=[0,80], title="PV module - P(I)", xaxis_title="I", yaxis_title="P"))

data: [
  "scatter with fields mode, name, type, x, and y",
  "scatter with fields mode, name, type, x, and y",
  "scatter with fields mode, name, type, x, and y",
  "scatter with fields mode, name, type, x, and y",
  "scatter with fields mode, name, type, x, and y"
]

layout: "layout with fields margin, template, title, xaxis, and yaxis"

For the characteristics depending on x, the axis have been swapped here. When plotting with the explicit function, there is a problem with the logarithm, which must not be less than zero. Therefore, the range of values must be very fine tuned. But since we can see in the upper two plots how steep the function is, this is very difficult.

### Select Test point

Here we use the functions that are also part of ElectricGrid.jl to evaluate the characteristic curves at selected points.

In [29]:
v_test = 20
i_test = get_I(ArrayPV, v_test, 1000, 27)

3.487780584916441

In [30]:
i_test2 = 13.951122339665764
v_test2 = get_V(ArrayPV, i_test2, 1000, 27)

0

In [31]:
Plot([scatter(x=V, y=i[:,5], mode="lines", name=labels[1]),
      scatter(x=[v_test], y=[i_test], mode="markers", name="Test point",marker=attr(size=10))],
Layout(xaxis_range=[0,30], yaxis_range=[0,5], title="PV module - I(V)", xaxis_title="V", yaxis_title="I"))

data: [
  "scatter with fields mode, name, type, x, and y",
  "scatter with fields marker, mode, name, type, x, and y"
]

layout: "layout with fields margin, template, title, xaxis, and yaxis"

In [ ]:
Plot(scatter(x=[1],y=[1], mode="markers"))

## Characteristics for PV arrays

In [ ]:
T = 25
G = collect(200:200:1000)
V = collect(0:1:500);

In [ ]:
i = GetI_vec(PV_arr, V, G, T);

In [ ]:
plot(V, i, xlim=(0,350),ylim=(0,20),title="PV array - I(V)", label=labels, xlabel="V", ylabel="I")
scatter!([v_test], [i_test], color="blue", markershape=:star5, label="Test")

In [ ]:
V_ = V.*ones(5)';
P = i .* V_;

In [ ]:
plot(V_, P, xlim=(0,300), ylim=(0,3500),title="PV array - P(V)", label=labels, xlabel="V", ylabel="P")

In [ ]:
plot(i, V_, xlim=(0,30), ylim=(0,300),title="PV array - V(I)", label=labels, xlabel="I", ylabel="V")
scatter!([i_test2],[v_test2], color="blue", markershape=:star5, label="Test")

Here you can see that the test point is not exactly on the line. This is due to the numerical inaccuracy. Since we are on the steep part, small deviations lead to larger shifts along the y-axis.

In [ ]:
plot(i, P, xlim=(0,20), ylim=(0,3500),title="PV array - V(I)", label=labels, xlabel="I", ylabel="P")